# Inaugural Project

Group: Slaget i Køge Bugt

Group members: Asger Bjerg Pedersen, vjr207. Rasmus Juel, qsw248.

Import and settings.

In [1]:
import numpy as np

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

**Labour supply problem**. Consider a consumer solving the following maximization problem

$$ c^\star, \ell^\star = \arg\max_{c, l}\log(c) - v\cdot\frac{\ell^{1+\frac{1}{\epsilon}}}{1+\frac{1}{\epsilon}} $$
s.t.
$$ x=m+w\ell-[\tau_0w\ell+\tau_1\max\{w\ell-\kappa,0\}] $$

$$ c\in[0,x] $$

$$ \ell\in[0,1] $$


where $c$ is consumption, $\ell$ is labour supply, $m$ is cash-on-hand, $w$ is the wage rate, $\tau_0$ is the standard labour income tax, $\tau_1$ is tge top bracket labour income tax, $\kappa$ is the cut-off ofr the top labour income bracket, $x$ is total resources, $v$ scales the disutility of labour, and $\epsilon$ is the (Frisch) elasticity of labour supply.

## Question 1

To construct a function solving the consumer maximization problem, we first define the parameters that enter the utility function and the resource constraint.

In [4]:
#1.1. Define value of variables
money = 1 #m
v = 10 #v
frisch = 0.3 #epsilon
ltax = 0.4 #tau_0
ttax = 0.1 #tau_1
cutoff = 0.4 #kappa
w = 0.5 #w set with an initial value

Now, define the labour supply utility function and the budget constraint function.

In [5]:
#1.2. Define utility as a function of labour supply and consumption
def labour_utility(l,c,frisch,v):
    u = np.log(c) - v*l**(1 + 1/frisch)/(1 + 1/frisch)
    return u

#1.3. Define budget constraint; feasible consumption given labour supply and wage
def budget_constraint(money,w,l,ltax,ttax,cutoff):
    budget = money + w*l - w*l*ltax - ttax*max((w*l - cutoff),0)
    return budget

As we have now defined the functions that we wish to maximise, we move to solving the labour supply problem. Note that we apply:
\
\
$$c^\star=x$$
\
i.e. utility is monotonically increasing in consumption, the consumer will always spend his entire budget $x$ on consumption $c$. We solve the agent's labour supply problem using the solver command imported from the $scipy$ module, plugging in the resource constraint. 
\
\
The objective function $value\_of\_choice\_budget$ returns the negative value of the utility following an arbitrary pair of $(c,l)$. Applying $c$ as a function of $\ell$, we construct a function $optimiser$ that finds the labour supply $\ell^\star$ which minimises this negative utility given the set of parameters and the wage level $w$, and the feasible level of consumption following this labour supply. As the total time endowment is 1, we also apply the bounds that $\ell\in[0,1]$.

In [6]:
from scipy import optimize

#1.4. Objective function returning negative utility, to minimize
def value_of_choice_budget(l,w,frisch,v,ltax,ttax,cutoff):
    c = budget_constraint(money,w,l,ltax,ttax,cutoff)
    return -labour_utility(l,c,frisch,v)

#1.5.1. Call the solver given the target from 1.4 and the constraint from 1.3.
def optimiser(w,frisch,v,ltax,ttax,cutoff,money):
    sol_case1 = optimize.minimize_scalar(
        value_of_choice_budget, method = 'bounded',
        bounds=(0,1), args = (w,frisch,v,ltax,ttax,cutoff))
    """ Solving the agent's problem of optimal labour supply and consumption.
    
    Args:
    
        w (float): Individual wage
        frisch (float): Frisch elasticity of labour supply
        v (float): Disutility of labour multiplier
        ltax (float): lower tax rate
        ttax (float): additional upper tax rate
        cutoff (float): cutoff between lower and upper tax rate
        money (float): initial money endowment
        
    Returns:
    
        (list): Optimal labour supply, feasible consumption, utility level from the two former
    
    """

#1.5.2. Unpack optimal labour supply, then return feasible consumption and implied individual utility
    lstar = sol_case1.x
    cstar = budget_constraint(money,w,lstar,ltax,ttax,cutoff)
    ustar = labour_utility(lstar,cstar,frisch,v)
    return [lstar,cstar,ustar]

We have thus now constructed a function which solves the consumer maximization problem.

## Question 2

In this question, we plot $\ell^\star$ and $c^\star$ as functions of the wage rate, $w$, going from 0.5 to 1.5. For this purpose, we import the $pyplot$ module. Rather than making a continous function for the whole range $w \in [0.5,1.5]$, we create a vector with 1000 values of $\ell^\star$ and $c^\star$ given $w$ (derived with our $optimiser$ function).
\
\
The results are as follows:


In [7]:
import matplotlib.pyplot as plt
plt.style.use("seaborn")

#2.1. Set number of observations
N=1000

#2.2.1. Generate vectors of optimal labour supply and feasible consumption given wage.
w_vec=np.linspace(0.5,1.5,N)
l_vec=np.empty(N)
c_vec=np.empty(N)
for i,w in enumerate(w_vec):
    l_c_bundle=optimiser(w,frisch,v,ltax,ttax,cutoff,money)
    #2.2.2. As the optimiser function returns a list of optimal labour supply and feasible consumption,
            #extract the relevant results. Copy them to the list of labour supply and consupmtion.
    l_vec[i]=l_c_bundle[0]
    c_vec[i]=l_c_bundle[1]

#2.3.1 Create the figure
fig = plt.figure(figsize=(10,4))

#2.3.2. Left plot; labour supply
ax_left = fig.add_subplot(1,2,1)
ax_left.plot(w_vec,l_vec)

ax_left.set_title('Optimal labour supply given wage')
ax_left.set_xlabel('$w$')
ax_left.set_ylabel('$l^\star$')
ax_left.grid(True)

#2.3.3. right plot; feasible consumption
ax_right = fig.add_subplot(1,2,2)
ax_right.plot(w_vec,c_vec)

ax_right.set_title('Optimal consumption given wage')
ax_right.set_xlabel('$w$')
ax_right.set_ylabel('$c^\star$')
ax_right.grid(True)

As expected, we see a jump down around the threshold for the top tax $\kappa=0.4$ for $w=1$ since we are here going from paying the low tax to pay a higher tax on our additional earnings. Here, the consumer will experience an increase in the marginal cost of consumption relative to leisure, for which reason, as can be seen on the panel on the right-hand side, he will increase his demand for leisure while his demand for consumption is stagnant. Consumption will be kept at a level of 1.24, for which a labour income of 0.4 is required given a tax rate of 0.4 and a money endowment of 1. For some interval, the marginal utility from consumption will be ruled out by the marginal utility of leisure, requiring a smaller labour supply when the wage per unit of labour increases. Decreasing the labour supply such that the wage is arbitrarily below 0.4, however, it will be optimal to marginally increase this labour supply until the point where the total wage is excactly 0.4. When the marginal utility from consumption once again equals the marginal utility from leisure, the consumer will gradually begin increasing his labour supply again. 
\
\
All this goes without saying that the standard microeconomic requirement of the marginal rate of substitution between consumption and leisure should equal the relative prices between the two is not defined at an income level of 0.4.

## Question 3

We now calculate the total tax revenue for the conditions specified in the problem. We first draw a vector of individual wages $w_i$ where $w_i\sim U((0.5),(1.5))$ for the 10.000 consumers. We denote the optimal choices given $w$ of individual $i$ as $\ell_i^\star$ and $c_i^\star$. We then calculate the total tax revenue. To do so, we define the function 'TotalTax', which, using the optimser from the previous problem, finds the tax payment for indivudal $i$ and then sums all of the payments to get the total tax revenue.
\
\
This reads:

In [10]:
#3.1. Set seed and draw 10.000 random wages given the distribution
np.random.seed(4600)
PopWageLarge = np.random.uniform(0.5,1.5,size=10000)


def TotalTax(PopWageVector,frisch,v,ltax,ttax,cutoff,money):
    """ Calculating total tax revenue for a vector of preference-invariant agents idiosyncratic in wages.
    
    Args:
    
        PopWageVector (array): Popular wages
        frisch (float): Frisch elasticity of labour supply
        v (float): Disutility of labour multiplier
        ltax (float): lower tax rate
        ttax (float): additional upper tax rate
        cutoff (float): cutoff between lower and upper tax rate
        money (float): initial money endowment
        
    Returns:
    
        TotTax (float): Total tax revenue
    
    """
    #3.2. Return an array for individual tax payments
    N=len(PopWageVector)
    PopTaxes=np.zeros((N))

    for i,w in enumerate(PopWageVector):
        #3.3.1. Return optimal labour supply given wage from the optimiser function
        Ind_optimum=optimiser(w,frisch,v,ltax,ttax,cutoff,money)
        IndLabour=Ind_optimum[0]
        #3.3.2. Return tax payment given the income yielded by optimal labour supply
        PopTaxes[i]=ltax*w*IndLabour+ttax*max(w*IndLabour-cutoff,0)
    #3.4. Sum over all tax payments
    TotTax=sum(PopTaxes)
    return TotTax

#3.5. Call total tax functions given the array of randomly drawn wages.
TotTax0 = TotalTax(PopWageLarge,frisch,v,ltax,ttax,cutoff,money)
print(f'The total tax revenue is {TotTax0:.1f}')

The total tax revenue is 1627.2


Given the parameters, this total tax revenue of 1,627 is also something we would expect, since the tax is a product of the wage rate, $w$, and the labour supply, $\ell$. If all the 10,000 citizens pay $w\cdot\ell$ in tax, and $\ell$ is around $0.4$ with the wage rate $w \in \{0.5,1.5\}$, this value seems to be a plausible result for the total tax revenue.

## Question 4

We find out what the tax revenue would be if instead $\epsilon = 0,1$ by calling the function from the previous question with a Frish elasticity of labour supply of 0,1.

In [22]:
#4.1. Define new value of \epsilon and call the TotalTax function given the 10.000 wage levels and new epsilon
InelasticFrisch = 0.1
TotTaxHiFrisch = TotalTax(PopWageLarge,InelasticFrisch,v,ltax,ttax,cutoff,money)
print(f'The total tax revenue is {TotTaxHiFrisch:.1f}')

The total tax revenue is 3191.2


We see that as the elasticity of labour supply decreases, the marginal rate of substitution between consumption and leisure changes for a given wage rate such that it is optimal for the consumer to supply more labour. And so, the tax revenue increases to 3,191. Almost doubling the tax revenue, the change in the tax revenue is qualitatively changing in the direction one would expect for a lower elasticity of labour supply.

## Question 5

We now consider a politician who wishes to maximize the tax revenue. We want to find the values of $\tau_0$, $\tau_1$ and $\kappa$ for this to be realised, and afterwards, report the tax revenue which we would expect to obtain from this. When optimising, we are maximising the tax revenue, i.e. the output of the TotalTax function from problem 3 with respect to the vector $taxes=\{\tau_0^\star,\tau_1^\star,\kappa^\star\}$. This is --- in principle --- done given the randomly drawn vector PopWagesLarge (wages for all 10.000 citizens), the elasticity of labour supply, the scale and the cash-on-hand as given parameters from the problem text.
\
\
Our initial values for the vector was $\{0.4, 0.1, 0.4\}$, but we have placed the initial guesses in the function closer to the right solution afterwards for the to reduce the time it takes the kernel to solve the problem.
\
Noting that the wage levels are randomly distributed, equivalent results for $taxes=\{\tau_0^\star,\tau_1^\star,\kappa^\star\}$ could be estimated for populations of 100 or 1.000. According to the law of large numbers, this should have only negligible effects to the optimal $\{\tau_0^\star,\tau_1^\star,\kappa^\star\}$, but greatly reduce the time for solving the problem. Below the code, an additional cell solving the problem for the full population is provided.

In [25]:
#5.1 Draw alternative vector 
PopWageSmall = np.random.uniform(0.5,1.5,size=100)
PopWageMedium = np.random.uniform(0.5,1.5,size=1000)


#5.2.1  Create a vector 'taxes' which includes 'ltax', 'ttax' and 'cutoff'
        #return the total tax for the 100, 1000 or 10000 people
def value_of_choice_tax(taxes,PopWage,frisch,v,money):
    
    ltax = taxes[0]
    ttax = taxes[1]
    cutoff = taxes[2]
    return -TotalTax(PopWage,frisch,v,ltax,ttax,cutoff,money)

#5.2.2  Define the 'taxOptimiser' function, find the tax maximising values of the vector 'taxes'.
def taxOptimiser(PopWage,frisch,v,money):
    """ Solving the government's problem in maximising tax revenue.
    
    Args:
    
        w (array): Popular wages
        frisch (float): Frisch elasticity of labour supply
        v (float): Disutility of labour multiplier
        money (float): initial money endowment
        
    Returns:
    
        (list): Optimal lower tax rate, optimal additional upper tax rate, 
                cutoff between upper and lower tax rate
    
    """
    # 5.2.3 Call solver
    initial_guess = [0.785,0.055,0.470]
    sol_case3 = optimize.minimize(
        value_of_choice_tax,initial_guess,method='Nelder-Mead',
        args=(PopWage,frisch,v,money))

    ltaxStar=sol_case3.x[0]
    ttaxStar=sol_case3.x[1]
    cutoffStar=sol_case3.x[2]

    #5.2.4 Print the solution   
    print(f'Optimal lower tax rate is {ltaxStar:.3f}')
    print(f'Optimal upper tax rate is {ttaxStar:.3f}')
    print(f'Optimal cutoff income is {cutoffStar:.3f}')
    
    return[ltaxStar,ttaxStar,cutoffStar]

print('Optimal taxes and estimated total tax revenue, N=100')
[ltaxStar,ttaxStar,cutoffStar]=taxOptimiser(PopWageSmall,frisch,v,money)
TotTaxSmall = TotalTax(PopWageLarge,frisch,v,ltaxStar,ttaxStar,cutoffStar,money)
print(f'Estimated total tax revenue is {TotTaxSmall:.1f}')

print('Optimal taxes and estimated total tax revenue, N=1000')
[ltaxStar,ttaxStar,cutoffStar]=taxOptimiser(PopWageMedium,frisch,v,money)
TotTaxMedium = TotalTax(PopWageLarge,frisch,v,ltaxStar,ttaxStar,cutoffStar,money)
print(f'Estimated total tax revenue is {TotTaxMedium:.1f}')

Optimal taxes and estimated total tax revenue, N=100
Optimal lower tax rate is 0.786
Optimal upper tax rate is 0.047
Optimal cutoff income is 0.530
Estimated total tax revenue is 2462.3
Optimal taxes and estimated total tax revenue, N=1000
Optimal lower tax rate is 0.785
Optimal upper tax rate is 0.052
Optimal cutoff income is 0.526
Estimated total tax revenue is 2462.3


In [24]:
print('Optimal taxes and estimated total tax revenue, full population')
[ltaxStar,ttaxStar,cutoffStar]=taxOptimiser(PopWageLarge,frisch,v,money)
TotTaxLarge = TotalTax(PopWageLarge,frisch,v,ltaxStar,ttaxStar,cutoffStar,money)
print(f'Estimated total tax revenue is {TotTaxLarge:.2f}')

Optimal taxes and estimated total tax revenue, full population
Optimal lower tax rate is 0.785
Optimal upper tax rate is 0.054
Optimal cutoff income is 0.531
Estimated total tax revenue is 2462.34


Our optimiser suggests that the optimal tax rate is 78.5 %, the additional tax rate for top incomes should be 5.4 %, so that their tax rate increases to 83.9 %, and the cutoff is increased to an income of 0,531. We also note that, even though there are minor differences in the parameters found for the population of 100, 1000 and 10000 citizens, the estimated total tax revenue is the same for all the populations, showing exactly our point that law of large numbers apply in this particular problem. The small differences in the parameters found just suggests that there are different combinations of the paramters which all together maximizes the total tax revenue.
\
\
The tax revenue for the parameters of the problem is so increased from 1627 to 2462, showing that the values (found with the optimiser) for the tax rates and the cutoff firmly increases the tax revenue.

# Conclusion

In this notebook, we have derived optimal labour supply given wages and tax parameters. Optimal labour supply is increasing in the wage per unit of labour supplied, however, distorting taxes lower the labour supply, which is especially clear when looking at those paying high income taxes. For a politician wishing to maximise the tax revenue, it is possible to increase the tax rates as well as the tax revenue substantially from the initial conditions.